In [1]:
import os
import logging
import spacy
import subjectVerbObjectExtractUtil as svoExtract
import experimentUtil as expUtil
import mergeGraphUtil as mergeUtil
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
logging.getLogger().setLevel(logging.INFO)


Bad key "margins.tight" on line 7 in
/Users/jfairbanks6/.matplotlib/stylelib/presentation.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
INTRO_MD_DIR = "introMDFiles"
CLEAN_INTRO_MD_DIR = "cleanedTexts"
SUBJECT_OBJECT_VERB_CSV = "svo.csv"

In [3]:
#Clean and Extract Text from Intro.md Files

for file in os.listdir(INTRO_MD_DIR):
    input_file_string = open(os.path.join(INTRO_MD_DIR, file)).read()
    proccessedText = svoExtract.extract(input_file_string)
    outputFileName, _ = os.path.splitext(file)
    outputFileName = os.path.join(CLEAN_INTRO_MD_DIR, outputFileName)
    outputFileName += ".txt"
    
    with open(outputFileName, 'w') as filetowrite:
        filetowrite.write(proccessedText)
        filetowrite.close()
        
    logging.info(outputFileName + " file created")

2019-03-25 10:59:00,490 cleanedTexts/semiparametric.txt file created
2019-03-25 10:59:00,492 cleanedTexts/khan.txt file created
2019-03-25 10:59:00,501 cleanedTexts/sis_intro.txt file created
2019-03-25 10:59:00,507 cleanedTexts/seird_stochastic.txt file created
2019-03-25 10:59:00,523 cleanedTexts/karlsson.txt file created
2019-03-25 10:59:00,539 cleanedTexts/ross.txt file created
2019-03-25 10:59:00,561 cleanedTexts/coalescent.txt file created
2019-03-25 10:59:00,571 cleanedTexts/mayanderson.txt file created
2019-03-25 10:59:00,575 cleanedTexts/deleo.txt file created
2019-03-25 10:59:00,594 cleanedTexts/sir_model_intro.txt file created
2019-03-25 10:59:00,615 cleanedTexts/lloydjansen.txt file created
2019-03-25 10:59:00,619 cleanedTexts/erlang.txt file created
2019-03-25 10:59:00,641 cleanedTexts/blackross.txt file created
2019-03-25 10:59:00,650 cleanedTexts/1host1vector.txt file created
2019-03-25 10:59:00,663 cleanedTexts/seir_intro.txt file created


In [4]:
#Extract Subject Verb Object
#Creates subject verb object csv file
svoExtract.extractSVO(CLEAN_INTRO_MD_DIR)

2019-03-25 10:59:00,738 writing 4 tuples
2019-03-25 10:59:00,786 writing 2 tuples
2019-03-25 10:59:00,863 writing 15 tuples
2019-03-25 10:59:00,889 writing 1 tuples
2019-03-25 10:59:00,911 writing 1 tuples
2019-03-25 10:59:00,950 writing 2 tuples
2019-03-25 10:59:01,046 writing 14 tuples
2019-03-25 10:59:01,101 writing 1 tuples
2019-03-25 10:59:01,224 writing 18 tuples
2019-03-25 10:59:01,269 writing 5 tuples
2019-03-25 10:59:01,306 writing 0 tuples
2019-03-25 10:59:01,345 writing 5 tuples
2019-03-25 10:59:01,387 writing 5 tuples
2019-03-25 10:59:01,458 writing 21 tuples
2019-03-25 10:59:01,487 writing 5 tuples
2019-03-25 10:59:01,489 File svo.csv created


In [5]:
import umap
umap.UMAP

umap.umap_.UMAP

In [6]:
# Creates CSV of UMAP and DBSCAN Clusters and InSample Vs OutSample Similarities
# Outputs clusteringLabels.csv

#expUtil.createCluster(SUBJECT_OBJECT_VERB_CSV)

In [7]:
# Get Kmeans Experiment Accuracy
# Assigns Cluster Labels to Variable Inputs to KMeansPredicted.csv and computes accuracy by taking the difference
# with the ground truth labels in JuliaDataVariable.csv
expUtil.runKMeansExp()

250
<class 'numpy.ndarray'> <class 'int'> <class 'numpy.ndarray'>


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mUnknown attribute 'shape' of type none
[1m
File "../../../../../anaconda/lib/python3.6/site-packages/umap/umap_.py", line 88:[0m
[1mdef smooth_knn_dist(distances, k, n_iter=64, local_connectivity=1.0, bandwidth=1.0):
    <source elided>
    target = np.log2(k) * bandwidth
[1m    rho = np.zeros(distances.shape[0])
[0m    [1m^[0m[0m
[0m
[0m[1m[1] During: typing of get attribute at /Users/jfairbanks6/anaconda/lib/python3.6/site-packages/umap/umap_.py (88)[0m
[1m
File "../../../../../anaconda/lib/python3.6/site-packages/umap/umap_.py", line 88:[0m
[1mdef smooth_knn_dist(distances, k, n_iter=64, local_connectivity=1.0, bandwidth=1.0):
    <source elided>
    target = np.log2(k) * bandwidth
[1m    rho = np.zeros(distances.shape[0])
[0m    [1m^[0m[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To s

In [ ]:
# Uses cluster that is most "similar" to the variable for cluster assignment
expUtil.runUMapSimilarityExp()

In [ ]:
# Uses KMeans for cluster assignment, computes the similarity of the assignment with nodes
# that also belong to the cluster assignment 
averageSimArray = expUtil.runCombinationExp()

In [ ]:
#Creates Final CSV to create metaGraph

#Minimum threshold to merge 2 nodes from extracted subject verb objects
ClusterThreshold = .85

#Minimum threshold to assign variable to a cluster
VariableThreshold = .5

mergeUtil.createFinalGraph(ClusterThreshold, VariableThreshold, averageSimArray)

In [ ]:
print 3